In [18]:
import praw
import numpy as np
from praw.models import MoreComments
import pandas as pd
from datetime import datetime
from praw.exceptions import RedditAPIException
import plotly.graph_objects as go
import yfinance

In [4]:
reddit = praw.Reddit(
    
)


In [5]:
df = pd.read_csv('labeled-data.csv',index_col=0)
positives = set(open('positive.txt').read().split())
negatives = set(open('negative.txt').read().split())

df

,$MSFT,$AAPL,$TSLA,$AMZN,$NFLX,$META,$NVDA,$AMD
Guysmarket,"['w69hk8', 'vaoptj', 'w91vth', 'vaevci']","['137z7xg', 'x6oms2', 'vfgi4j', 'z68fnt', 'vfy...","['12s8cfb', 'vaevci']","['vaoptj', 'vaevci']",['w3cwti'],NaN,"['13ne5be', 'yx6rl6', 'yx0gth']",['wexhxz']
predictany007,"['13igt6u', 'w9cih5', '12zedri', '13j5014', '1...","['y83owq', '13rl2eq', '136w8h8', '138ikrk', 'z...","['13i691q', '12rq3pf', '12svh8b', '120qykr', '...","['13qudsg', '13rl2eq', 'z122gg', 'zya1j1', 'yv...","['12rqd26', '13pwxdb', '13l69lg', '10gdagf', '...","['13ppjis', 'yldmuu', 'v44p5o']",NaN,"['xy1yx2', 'weh5zm']"
WickedSensitiveCrew,"['13ia8cg', '1154egt', 'zrouhg', 'zvzi00', 'v8...","['12qwfjn', 'zvzi00', '10jckme', 'zbtscx', 'yd...",['12ekzkx'],"['zvzi00', 'zbq9l8']",NaN,['zvzi00'],NaN,NaN
DaddyDersch,"['12yvp9l', '12yvqq0', '10kh5rn']","['xsd18z', 'yqwe59', 'xqrdms', 'xouv0s', 'yjkj...","['12r45cv', '12s9361', '1144ms6', '11a9amh', '...",NaN,"['12r45cv', '12r46qm']","['12zyzqv', '12zzagw']","['13r0uu2', '13r0wga']","['vz5ejc', 'vzz7j9', 'w29gw7', 'w29flo', 'vz5d..."
msaleem,"['10s88q8', '10tv0xs', '10t0lwx']","['12buzi8', '10hueqy', '10lvspw', '10tv0xs', '...",NaN,"['11xfwwa', '10hueqy', '1035yg6', '10tv0xs', '...","['10juhvh', '1097x21']","['13m5yke', '11djv5g', '11f7pih', '1027i7o', '...",NaN,['10root9']
bwang29,['yf8b1n'],['yf8b1n'],NaN,NaN,['yf8b1n'],['yf8b1n'],NaN,['yf8b1n']
Puginator,['10wajfm'],"['12pjlnd', '124zgnb', '10pkib2', '11zt7gt']","['10b923r', '13ruc5g', '129rlfw', '11frmg2', '...","['11wky2r', '11q9s6w', '10mzeth', '10keumq', '...",NaN,"['11r62p6', '10r6b7j', '1194tcj', '10qax3a']",NaN,['10qb2lq']
_hiddenscout,"['108b31n', '11dha3y', '10jghuc']","['12qorcb', '10okvff']","['12u12hf', '10amo8n']","['12e3diy', '11dha3y', '10okvff']",['12r2j2b'],NaN,NaN,['v18lmi']
GonjaNinja420,NaN,"['yehgd0', 'xqf3hg']",NaN,['xoksim'],['xnefh3'],NaN,NaN,"['xns17k', 'xnx1v8', 'xoic3p']"
Wilingaway,NaN,"['x3at6w', 'wpxt0p']",NaN,['wpxt0p'],NaN,['x3at6w'],NaN,['viyn5d']


In [6]:
def post_sentiment(text):
    
    p = len([w for w in text if w in positives])
    n = len([w for w in text if w in negatives])
    
    if n == 0:
        return 0.0
    
    return round(p / n, 3)

In [10]:
post = reddit.submission(id='wj8gj3')
title = post.title
body = post.selftext

In [11]:
def post_to_dict(post_id):
    
    post = reddit.submission(id=post_id)
    date = datetime.fromtimestamp(post.created_utc)
    text = (post.title + post.selftext).split()
    sentiment = post_sentiment(text)
    post_dict = {'Title' : post.title, 'Author' : post.author, 'Subreddit' : post.subreddit, 'Date': date, 'Sentiment' : sentiment}
    return post_dict

In [21]:
example = post_to_dict('w3cwti')

In [74]:
tsla = yfinance.Ticker('TSLA')
hist = tsla.history(period='18mo')

In [85]:
timestamp = example['Date']
closest_timestamp = hist.index[hist.index.get_loc(timestamp, method='nearest')]
fig = go.Figure(data=go.Scatter(x=hist.index, y=hist['Close'], mode='lines'))
specific_close_value = hist.loc[closest_timestamp, 'Close']

C:\Users\azras\AppData\Local\Temp\ipykernel_9388\1122170248.py:2: FutureWarning:

Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.

C:\Users\azras\AppData\Local\Temp\ipykernel_9388\1122170248.py:2: FutureWarning:

Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.



In [86]:
sen_color = 'red'
if example['Sentiment'] >= 1:
    sen_color = 'green'


In [87]:
fig.add_trace(go.Scatter(x=[closest_timestamp], y=[specific_close_value], mode='markers', marker=dict(size=8, color=sen_color)))

In [88]:
fig.show()

# Conclusion

If the user talked about the stock in a positive light, it would be marked as green. If it was negative, it was marked as red. This is just the one example to treat reddit users as "investors" to see how accurate their "recommendations" are. Due to Reddit's API changes at the time of development, I was unable to apply this throughout the entire dataframe. 